In [1]:
#Load required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import glob
import re
import emoji
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
# import imblearn
# from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import nltk
from nltk.corpus import wordnet
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

from sklearn import preprocessing

In [2]:
#Loading the raw data

In [3]:
import pandas as pd
df=pd.read_excel("./BotReliabilityRaw.xlsx")
df

,Group,Tweet,RA1 Intent,RA2 Intent,Final Intent
0,1,I wanna say a dollar more even being just righ...,others,others,others
1,1,Hope your having a good day Gidgett. Have a wo...,support,support,support
2,1,I will take deep breaths find a great hobby to...,others,others,others
3,1,Hope everyone is doing great I found out the o...,support,support,support
4,1,Go to Walmart & pick up an empty spray bottle ...,others,others,others
...,...,...,...,...,...
82043,X8,oh I feel you on the morning coffee not being ...,cravings,cravings,cravings
82044,X8,That's awesome!!!!! :),support,others,others
82045,X8,so far it's been a good day. no strong craving...,smokefree,smokefree,smokefree
82046,X8,I know. Me too. Its just so cold here!! Time t...,weightGain,weightGain,weightGain


In [4]:
df.rename(columns={'Final Intent': 'intent'}, inplace=True)
df['intent'] = df['intent'].str.strip().str.lower()
df = df[['Group','Tweet','intent']]
df

,Group,Tweet,intent
0,1,I wanna say a dollar more even being just righ...,others
1,1,Hope your having a good day Gidgett. Have a wo...,support
2,1,I will take deep breaths find a great hobby to...,others
3,1,Hope everyone is doing great I found out the o...,support
4,1,Go to Walmart & pick up an empty spray bottle ...,others
...,...,...,...
82043,X8,oh I feel you on the morning coffee not being ...,cravings
82044,X8,That's awesome!!!!! :),others
82045,X8,so far it's been a good day. no strong craving...,smokefree
82046,X8,I know. Me too. Its just so cold here!! Time t...,weightgain


In [5]:
#Getting rid of Nan Values

In [6]:
df=df.dropna().reset_index(drop=True)
df

,Group,Tweet,intent
0,1,I wanna say a dollar more even being just righ...,others
1,1,Hope your having a good day Gidgett. Have a wo...,support
2,1,I will take deep breaths find a great hobby to...,others
3,1,Hope everyone is doing great I found out the o...,support
4,1,Go to Walmart & pick up an empty spray bottle ...,others
...,...,...,...
81862,X8,oh I feel you on the morning coffee not being ...,cravings
81863,X8,That's awesome!!!!! :),others
81864,X8,so far it's been a good day. no strong craving...,smokefree
81865,X8,I know. Me too. Its just so cold here!! Time t...,weightgain


In [7]:
duplicates = df[df.duplicated(subset=['Tweet'], keep=False)]
duplicates

,Group,Tweet,intent
43,1,Good morning ladies have a great day,greetings
44,1,Good morning ladies have a great day,greetings
45,1,Good morning ladies have a great day,greetings
56,1,you too Gidgett,others
59,1,Good morning Gidgett,greetings
...,...,...,...
81782,X8,That's wonderful!,others
81783,X8,me too,others
81790,X8,yes.,others
81792,X8,Thank you.,others


In [8]:
duplicates=duplicates.reset_index(drop=True)
for i in range(0,100):
    print(duplicates['Tweet'][i])

Good morning ladies have a great day
Good morning ladies have a great day
Good morning ladies have a great day
you too Gidgett
Good morning Gidgett
Congrats
Good morning Cindy
really cute I've got two kitties one named frisky and one named snowball
Good morning ladies!
Your very welcome!
lol!
Yes it is
Good morning ladies have a great day.
Good morning ladies have a great day.
your welcome
#NewProfilePic
same here
I'm here
You're doing great
Thank you!
awww
Thanks Cindy you too
I agree totally
Thank you
Good morning ladies have a great day
Me three
thanks
Good morning ladies have a great day
Good morning ladies have a great day
Good morning! :-)
You're doing great!
Good morning ladies have a great day
I hope you feel better soon
Thank you Cindy you also
Yay
Good morning ladies have a great day
Good morning everyone. Have a great day!
Good morning ladies have a great day
Good morning ladies have a great day.
Thank you..
Thanks Starla same to you.
You too Gidgett
Good morning beautiful l

In [9]:
df=df.drop_duplicates(subset='Tweet')
df=df.reset_index(drop=True)
df

,Group,Tweet,intent
0,1,I wanna say a dollar more even being just righ...,others
1,1,Hope your having a good day Gidgett. Have a wo...,support
2,1,I will take deep breaths find a great hobby to...,others
3,1,Hope everyone is doing great I found out the o...,support
4,1,Go to Walmart & pick up an empty spray bottle ...,others
...,...,...,...
77351,X8,oh I feel you on the morning coffee not being ...,cravings
77352,X8,That's awesome!!!!! :),others
77353,X8,so far it's been a good day. no strong craving...,smokefree
77354,X8,I know. Me too. Its just so cold here!! Time t...,weightgain


In [10]:
#Dividing test set and rest of the intent set

In [11]:
import pandas as pd
df_check=df
# Ensure the group column is treated correctly
df_check.iloc[:, 0] = pd.to_numeric(df.iloc[:, 0], errors='coerce')  # Convert numeric, set non-numeric to NaN

# Filter test set (groups 30 to 36)
test = df[df_check.iloc[:, 0].between(30, 36, inclusive="both")]

# Rest of the data (everything else)
intents = df[~df.index.isin(test.index)]

# Display results
print(f"Test Set Size: {test.shape[0]}")
print(f"Intent Set Size: {intents.shape[0]}")


Test Set Size: 15207
Intent Set Size: 62149


In [12]:
od_df=pd.read_excel("./BotTesting_Extra_to Shagoto.xlsx")
od_df

,Tweet,Correct Intent
0,Step 1 is too strong makes me sick,NRT_OD
1,Using the patch now but too potent,NRT_OD
2,Step 1 was too strong and made me nauseous and...,NRT_OD
3,The high dose was too much was having an overdose,NRT_OD
4,Don't smoke with the patch you can get sick,NRT_OD
5,I feel better after changing to a lower patch,NRT_OD
6,I changed to the lower patch because it was to...,NRT_OD
7,The patch might be too strong for you,NRT_OD
8,I found out the patch was too strong for my bo...,NRT_OD
9,I alternate gum with regular mints so it isn't...,NRT_OD


In [13]:
od_df.rename(columns={'Correct Intent': 'intent'}, inplace=True)
od_df['intent'] = od_df['intent'].str.strip().str.lower()
od_df = od_df[['Tweet','intent']]
od_df

,Tweet,intent
0,Step 1 is too strong makes me sick,nrt_od
1,Using the patch now but too potent,nrt_od
2,Step 1 was too strong and made me nauseous and...,nrt_od
3,The high dose was too much was having an overdose,nrt_od
4,Don't smoke with the patch you can get sick,nrt_od
5,I feel better after changing to a lower patch,nrt_od
6,I changed to the lower patch because it was to...,nrt_od
7,The patch might be too strong for you,nrt_od
8,I found out the patch was too strong for my bo...,nrt_od
9,I alternate gum with regular mints so it isn't...,nrt_od


In [14]:
duplicates = od_df[od_df.duplicated(subset=['Tweet'], keep=False)]
duplicates

,Tweet,intent


In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

train_od, test_od = train_test_split(od_df, test_size=0.2, random_state=42)

# Append split data
intents = pd.concat([intents, train_od], ignore_index=True)
test = pd.concat([test, test_od], ignore_index=True)



In [16]:
intents

,Group,Tweet,intent
0,1.0,I wanna say a dollar more even being just righ...,others
1,1.0,Hope your having a good day Gidgett. Have a wo...,support
2,1.0,I will take deep breaths find a great hobby to...,others
3,1.0,Hope everyone is doing great I found out the o...,support
4,1.0,Go to Walmart & pick up an empty spray bottle ...,others
...,...,...,...
62192,NaN,The lozenges have a very strong taste of nicot...,nrt_od
62193,NaN,I'm definently oding on these things,nrt_od
62194,NaN,Does anyone else have a potency problem with t...,nrt_od
62195,NaN,Is there a way to get less nicotine from the l...,nrt_od


In [17]:
test

,Group,Tweet,intent
0,30.0,Celebrate You\n\nStayingÂ smokefreeÂ is a big ...,support
1,30.0,Busy i took a nap after my girls got their hai...,others
2,30.0,You got this just keep your gum close by and m...,support
3,30.0,Q: How do you keep your husband from reading y...,others
4,30.0,Hot and humid yuck,others
...,...,...,...
15214,NaN,I am going to talk to my doctor to go to step ...,nrt_od
15215,NaN,I skipped the step 1 patches because I couldn'...,nrt_od
15216,NaN,These patches are too powerful,nrt_od
15217,NaN,Are you using the lozenges too? Don't you thin...,nrt_od


In [18]:
duplicates = intents[intents.duplicated(subset=['Tweet'], keep=False)]
duplicates

,Group,Tweet,intent


In [19]:
duplicates = test[test.duplicated(subset=['Tweet'], keep=False)]
duplicates

,Group,Tweet,intent


In [20]:
matching_vals = intents['Tweet'].isin(test['Tweet'])
matching_vals

0        False
1        False
2        False
3        False
4        False
         ...  
62192    False
62193    False
62194    False
62195    False
62196    False
Name: Tweet, Length: 62197, dtype: bool

In [21]:
print("Matching values:", intents['Tweet'][matching_vals].tolist())

Matching values: []


In [22]:
matching_vals = test['Tweet'].isin(intents['Tweet'])
matching_vals

0        False
1        False
2        False
3        False
4        False
         ...  
15214    False
15215    False
15216    False
15217    False
15218    False
Name: Tweet, Length: 15219, dtype: bool

In [23]:
print("Matching values:", test['Tweet'][matching_vals].tolist())

Matching values: []


In [24]:
#Renaming columns-->intent

In [25]:
intents["intent"]=intents.intent.str.lower().str.replace("'|’","")
intents["intent"]=np.where(intents.intent=="dreams","nrt_dreams",\
                  np.where(intents.intent=="smokeless","smokingless",\
                  np.where(intents.intent=="others","noresponse",\
                  np.where(intents.intent=="costs","savingmoney",intents.intent))))

# # Converting the codes to appropriate categories using a dictionary

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

intents['encoded_intent'] = le.fit_transform(intents['intent'].tolist())
print(intents['intent'].groupby(intents['encoded_intent']).unique().apply(pd.Series))

print('size',intents.groupby(["intent"]).size())

intents.head()

                                  0
encoded_intent                     
0                          cigsmell
1                          cravings
2                             ecigs
3                              fail
4                         greetings
5                            health
6                        noresponse
7                     nrt_don'twork
8                        nrt_dreams
9                      nrt_howtouse
10                      nrt_itworks
11              nrt_mouthirritation
12                     nrt_nauseous
13                           nrt_od
14               nrt_skinirritation
15                   nrt_stickissue
16                         quitdate
17                      savingmoney
18                           scared
19                        smokefree
20                      smokingless
21                           stress
22                          support
23                        tiredness
24                       weightgain
size intent
cigsmell        

,Group,Tweet,intent,encoded_intent
0,1.0,I wanna say a dollar more even being just righ...,noresponse,6
1,1.0,Hope your having a good day Gidgett. Have a wo...,support,22
2,1.0,I will take deep breaths find a great hobby to...,noresponse,6
3,1.0,Hope everyone is doing great I found out the o...,support,22
4,1.0,Go to Walmart & pick up an empty spray bottle ...,noresponse,6


In [26]:
intents['intent'].value_counts()

intent
noresponse             34791
support                 8440
smokefree               4096
greetings               2438
cravings                1928
quitdate                1706
health                  1130
fail                    1032
nrt_itworks              820
weightgain               708
stress                   677
savingmoney              607
nrt_howtouse             472
cigsmell                 437
nrt_don'twork            409
scared                   391
smokingless              380
nrt_dreams               344
ecigs                    273
nrt_stickissue           251
nrt_skinirritation       206
tiredness                204
nrt_mouthirritation      186
nrt_nauseous             163
nrt_od                   108
Name: count, dtype: int64

In [27]:
only_noresponse=intents[intents['intent']=='noresponse']
only_noresponse

,Group,Tweet,intent,encoded_intent
0,1.0,I wanna say a dollar more even being just righ...,noresponse,6
2,1.0,I will take deep breaths find a great hobby to...,noresponse,6
4,1.0,Go to Walmart & pick up an empty spray bottle ...,noresponse,6
6,1.0,I was scared it would come off..lol.. I'll try...,noresponse,6
8,1.0,it also looks like I haven't tweeted in days. ...,noresponse,6
...,...,...,...,...
62138,NaN,yeah.,noresponse,6
62142,NaN,"NYC is great. Idk if I'd want to live there, t...",noresponse,6
62143,NaN,I feel like I want to give up this supervisor ...,noresponse,6
62145,NaN,That's awesome!!!!! :),noresponse,6


In [28]:
only_noresponse=only_noresponse.reset_index(drop=True)
only_noresponse

,Group,Tweet,intent,encoded_intent
0,1.0,I wanna say a dollar more even being just righ...,noresponse,6
1,1.0,I will take deep breaths find a great hobby to...,noresponse,6
2,1.0,Go to Walmart & pick up an empty spray bottle ...,noresponse,6
3,1.0,I was scared it would come off..lol.. I'll try...,noresponse,6
4,1.0,it also looks like I haven't tweeted in days. ...,noresponse,6
...,...,...,...,...
34786,NaN,yeah.,noresponse,6
34787,NaN,"NYC is great. Idk if I'd want to live there, t...",noresponse,6
34788,NaN,I feel like I want to give up this supervisor ...,noresponse,6
34789,NaN,That's awesome!!!!! :),noresponse,6


In [29]:
only_noresponse.to_csv("only_noresponse_train_and_eval.csv")

In [ ]:
#downsampling noresponse class

In [27]:
a=intents['intent'].value_counts()
s=a[1:]
means=s.mean()
s

intent
support                8440
smokefree              4096
greetings              2438
cravings               1928
quitdate               1706
health                 1130
fail                   1032
nrt_itworks             820
weightgain              708
stress                  677
savingmoney             607
nrt_howtouse            472
cigsmell                437
nrt_don'twork           409
scared                  391
smokingless             380
nrt_dreams              344
ecigs                   273
nrt_stickissue          251
nrt_skinirritation      206
tiredness               204
nrt_mouthirritation     186
nrt_nauseous            163
nrt_od                  108
Name: count, dtype: int64

In [29]:
means

1141.9166666666667

In [30]:
# Step 1: Filter "noresponse" to get the first 5400 instances
noresponse_subset = intents[intents['intent'] == 'noresponse'].iloc[:1142]

# Step 2: Get all rows where intent is not "noresponse"
other_intents = intents[intents['intent'] != 'noresponse']

# Step 3: Combine the filtered "noresponse" and other intents
filtered_intents = pd.concat([noresponse_subset, other_intents], ignore_index=True)

In [31]:
noresponse_subset['intent'].value_counts()

intent
noresponse    1142
Name: count, dtype: int64

In [32]:
other_intents['intent'].value_counts()

intent
support                8440
smokefree              4096
greetings              2438
cravings               1928
quitdate               1706
health                 1130
fail                   1032
nrt_itworks             820
weightgain              708
stress                  677
savingmoney             607
nrt_howtouse            472
cigsmell                437
nrt_don'twork           409
scared                  391
smokingless             380
nrt_dreams              344
ecigs                   273
nrt_stickissue          251
nrt_skinirritation      206
tiredness               204
nrt_mouthirritation     186
nrt_nauseous            163
nrt_od                  108
Name: count, dtype: int64

In [33]:
filtered_intents['intent'].value_counts() 

intent
support                8440
smokefree              4096
greetings              2438
cravings               1928
quitdate               1706
noresponse             1142
health                 1130
fail                   1032
nrt_itworks             820
weightgain              708
stress                  677
savingmoney             607
nrt_howtouse            472
cigsmell                437
nrt_don'twork           409
scared                  391
smokingless             380
nrt_dreams              344
ecigs                   273
nrt_stickissue          251
nrt_skinirritation      206
tiredness               204
nrt_mouthirritation     186
nrt_nauseous            163
nrt_od                  108
Name: count, dtype: int64

In [34]:
intents=filtered_intents
intents

,Group,Tweet,intent,encoded_intent
0,1.0,I wanna say a dollar more even being just righ...,noresponse,6
1,1.0,I will take deep breaths find a great hobby to...,noresponse,6
2,1.0,Go to Walmart & pick up an empty spray bottle ...,noresponse,6
3,1.0,I was scared it would come off..lol.. I'll try...,noresponse,6
4,1.0,it also looks like I haven't tweeted in days. ...,noresponse,6
...,...,...,...,...
28543,NaN,The lozenges have a very strong taste of nicot...,nrt_od,13
28544,NaN,I'm definently oding on these things,nrt_od,13
28545,NaN,Does anyone else have a potency problem with t...,nrt_od,13
28546,NaN,Is there a way to get less nicotine from the l...,nrt_od,13


In [35]:
##Split the train and eval

In [36]:
train_df, val_df = train_test_split(
    intents,  # Split the entire DataFrame
    test_size=0.25,  # 25% for validation
    random_state=42,  # Reproducibility
    stratify=intents['encoded_intent']  # Ensure class balance
)

In [37]:
train_df['intent'].value_counts()

intent
support                6330
smokefree              3072
greetings              1828
cravings               1446
quitdate               1280
noresponse              856
health                  848
fail                    774
nrt_itworks             615
weightgain              531
stress                  508
savingmoney             455
nrt_howtouse            354
cigsmell                328
nrt_don'twork           307
scared                  293
smokingless             285
nrt_dreams              258
ecigs                   205
nrt_stickissue          188
nrt_skinirritation      154
tiredness               153
nrt_mouthirritation     140
nrt_nauseous            122
nrt_od                   81
Name: count, dtype: int64

In [38]:
val_df['intent'].value_counts()

intent
support                2110
smokefree              1024
greetings               610
cravings                482
quitdate                426
noresponse              286
health                  282
fail                    258
nrt_itworks             205
weightgain              177
stress                  169
savingmoney             152
nrt_howtouse            118
cigsmell                109
nrt_don'twork           102
scared                   98
smokingless              95
nrt_dreams               86
ecigs                    68
nrt_stickissue           63
nrt_skinirritation       52
tiredness                51
nrt_mouthirritation      46
nrt_nauseous             41
nrt_od                   27
Name: count, dtype: int64

In [39]:
#Renaming test intents

In [30]:
test["intent"]=test.intent.str.lower().str.replace("'|’","")
test["intent"]=np.where(test.intent=="dreams","nrt_dreams",\
                  np.where(test.intent=="smokeless","smokingless",\
                  np.where(test.intent=="others","noresponse",\
                  np.where(test.intent=="costs","savingmoney",test.intent))))

# # Converting the codes to appropriate categories using a dictionary

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
test['encoded_intent'] = le.fit_transform(test['intent'].tolist())
print(test['intent'].groupby(test['encoded_intent']).unique().apply(pd.Series))
test.head()

                                  0
encoded_intent                     
0                          cigsmell
1                          cravings
2                             ecigs
3                              fail
4                         greetings
5                            health
6                        noresponse
7                     nrt_don'twork
8                        nrt_dreams
9                      nrt_howtouse
10                      nrt_itworks
11              nrt_mouthirritation
12                     nrt_nauseous
13                           nrt_od
14               nrt_skinirritation
15                   nrt_stickissue
16                         quitdate
17                      savingmoney
18                           scared
19                        smokefree
20                      smokingless
21                           stress
22                          support
23                        tiredness
24                       weightgain


,Group,Tweet,intent,encoded_intent
0,30.0,Celebrate You\n\nStayingÂ smokefreeÂ is a big ...,support,22
1,30.0,Busy i took a nap after my girls got their hai...,noresponse,6
2,30.0,You got this just keep your gum close by and m...,support,22
3,30.0,Q: How do you keep your husband from reading y...,noresponse,6
4,30.0,Hot and humid yuck,noresponse,6


In [31]:
test=test.reset_index(drop=True)
test

,Group,Tweet,intent,encoded_intent
0,30.0,Celebrate You\n\nStayingÂ smokefreeÂ is a big ...,support,22
1,30.0,Busy i took a nap after my girls got their hai...,noresponse,6
2,30.0,You got this just keep your gum close by and m...,support,22
3,30.0,Q: How do you keep your husband from reading y...,noresponse,6
4,30.0,Hot and humid yuck,noresponse,6
...,...,...,...,...
15214,NaN,I am going to talk to my doctor to go to step ...,nrt_od,13
15215,NaN,I skipped the step 1 patches because I couldn'...,nrt_od,13
15216,NaN,These patches are too powerful,nrt_od,13
15217,NaN,Are you using the lozenges too? Don't you thin...,nrt_od,13


In [32]:
test['intent'].value_counts()

intent
noresponse             9664
support                1983
smokefree               818
greetings               490
quitdate                318
cravings                267
health                  265
weightgain              167
fail                    152
stress                  148
nrt_itworks             135
nrt_howtouse            105
savingmoney              90
nrt_stickissue           83
scared                   83
cigsmell                 76
ecigs                    73
nrt_don'twork            72
nrt_dreams               54
tiredness                43
nrt_skinirritation       35
smokingless              33
nrt_nauseous             23
nrt_mouthirritation      21
nrt_od                   21
Name: count, dtype: int64

In [33]:
test_only_noresponse=test[test['intent']=='noresponse']
test_only_noresponse

,Group,Tweet,intent,encoded_intent
1,30.0,Busy i took a nap after my girls got their hai...,noresponse,6
3,30.0,Q: How do you keep your husband from reading y...,noresponse,6
4,30.0,Hot and humid yuck,noresponse,6
5,30.0,Q: What do you call a woman without an asshole...,noresponse,6
7,30.0,Those are great reasons!,noresponse,6
...,...,...,...,...
15184,36.0,We now have German shepherd poodle puppies,noresponse,6
15187,36.0,What's the update with you ladies? How's it go...,noresponse,6
15188,36.0,Who smokes when they drive a lot? I think that...,noresponse,6
15189,36.0,With the new day comes new strength and new th...,noresponse,6


In [34]:
test_only_noresponse=test_only_noresponse.reset_index(drop=True)
test_only_noresponse

,Group,Tweet,intent,encoded_intent
0,30.0,Busy i took a nap after my girls got their hai...,noresponse,6
1,30.0,Q: How do you keep your husband from reading y...,noresponse,6
2,30.0,Hot and humid yuck,noresponse,6
3,30.0,Q: What do you call a woman without an asshole...,noresponse,6
4,30.0,Those are great reasons!,noresponse,6
...,...,...,...,...
9659,36.0,We now have German shepherd poodle puppies,noresponse,6
9660,36.0,What's the update with you ladies? How's it go...,noresponse,6
9661,36.0,Who smokes when they drive a lot? I think that...,noresponse,6
9662,36.0,With the new day comes new strength and new th...,noresponse,6


In [35]:
test_only_noresponse.to_csv("test_only_noresponse.csv")

In [43]:
a=test['intent'].value_counts()
s=a[1:]
s.mean()

231.45833333333334

In [44]:
# Step 1: Filter "noresponse" to get the first 5400 instances
noresponse_subset = test[test['intent'] == 'noresponse'].iloc[:231]

# Step 2: Get all rows where intent is not "noresponse"
other_intents = test[test['intent'] != 'noresponse']

# Step 3: Combine the filtered "noresponse" and other intents
filtered_intents = pd.concat([noresponse_subset, other_intents], ignore_index=True)

In [45]:
noresponse_subset['intent'].value_counts() 

intent
noresponse    231
Name: count, dtype: int64

In [46]:
other_intents['intent'].value_counts() 

intent
support                1983
smokefree               818
greetings               490
quitdate                318
cravings                267
health                  265
weightgain              167
fail                    152
stress                  148
nrt_itworks             135
nrt_howtouse            105
savingmoney              90
nrt_stickissue           83
scared                   83
cigsmell                 76
ecigs                    73
nrt_don'twork            72
nrt_dreams               54
tiredness                43
nrt_skinirritation       35
smokingless              33
nrt_nauseous             23
nrt_mouthirritation      21
nrt_od                   21
Name: count, dtype: int64

In [47]:
filtered_intents['intent'].value_counts() 

intent
support                1983
smokefree               818
greetings               490
quitdate                318
cravings                267
health                  265
noresponse              231
weightgain              167
fail                    152
stress                  148
nrt_itworks             135
nrt_howtouse            105
savingmoney              90
nrt_stickissue           83
scared                   83
cigsmell                 76
ecigs                    73
nrt_don'twork            72
nrt_dreams               54
tiredness                43
nrt_skinirritation       35
smokingless              33
nrt_nauseous             23
nrt_mouthirritation      21
nrt_od                   21
Name: count, dtype: int64

In [48]:
test=filtered_intents
test

,Group,Tweet,intent,encoded_intent
0,30.0,Busy i took a nap after my girls got their hai...,noresponse,6
1,30.0,Q: How do you keep your husband from reading y...,noresponse,6
2,30.0,Hot and humid yuck,noresponse,6
3,30.0,Q: What do you call a woman without an asshole...,noresponse,6
4,30.0,Those are great reasons!,noresponse,6
...,...,...,...,...
5781,NaN,I am going to talk to my doctor to go to step ...,nrt_od,13
5782,NaN,I skipped the step 1 patches because I couldn'...,nrt_od,13
5783,NaN,These patches are too powerful,nrt_od,13
5784,NaN,Are you using the lozenges too? Don't you thin...,nrt_od,13


In [49]:
train_df=train_df.reset_index(drop=True)
train_df=train_df[['Tweet','intent']]
train_df

,Tweet,intent
0,I love running but have to stop to catch my br...,weightgain
1,Me too. Some days like when I am working I do ...,cravings
2,I was spending 20 bucks a day at my high point...,savingmoney
3,Hope everyone is having a good day,support
4,Hey guys hope everyone is having a wonderful l...,smokefree
...,...,...
21406,I know we can all do this!!! Tomarrow I am goi...,support
21407,We are here for you!,support
21408,I promised my son I would quit.,support
21409,Day 3 still going strong,smokefree


In [50]:
val_df=val_df.reset_index(drop=True)
val_df=val_df[['Tweet','intent']]
val_df

,Tweet,intent
0,Hello everyone . The closer Christmas it gets ...,stress
1,Oh no! I hope he has a quick recovery!,support
2,Good Morning & Happy Easter! Hope everyone is ...,support
3,Ok I'm back again. I'm actually only on day 5;...,smokefree
4,"Just made a very healthy lunch, did some yoga,...",smokefree
...,...,...
7132,they asked us to use our own twitter account f...,noresponse
7133,Glad I don't smoke anymore,smokefree
7134,Good morning!!! I'm feeling good. My quitDate ...,smokefree
7135,How are you Erica?When is your quitDate?,quitdate


In [51]:
test=test.reset_index(drop=True)
test=test[['Tweet','intent']]
test

,Tweet,intent
0,Busy i took a nap after my girls got their hai...,noresponse
1,Q: How do you keep your husband from reading y...,noresponse
2,Hot and humid yuck,noresponse
3,Q: What do you call a woman without an asshole...,noresponse
4,Those are great reasons!,noresponse
...,...,...
5781,I am going to talk to my doctor to go to step ...,nrt_od
5782,I skipped the step 1 patches because I couldn'...,nrt_od
5783,These patches are too powerful,nrt_od
5784,Are you using the lozenges too? Don't you thin...,nrt_od


In [52]:
train_df.to_csv("nrt_training_with_some_nores_v2.csv")
val_df.to_csv("nrt_validation_with_some_nores_v2.csv")
test.to_csv("nrt_test_with_some_nores_v2.csv")

In [53]:
train_df['intent'].value_counts()

intent
support                6330
smokefree              3072
greetings              1828
cravings               1446
quitdate               1280
noresponse              856
health                  848
fail                    774
nrt_itworks             615
weightgain              531
stress                  508
savingmoney             455
nrt_howtouse            354
cigsmell                328
nrt_don'twork           307
scared                  293
smokingless             285
nrt_dreams              258
ecigs                   205
nrt_stickissue          188
nrt_skinirritation      154
tiredness               153
nrt_mouthirritation     140
nrt_nauseous            122
nrt_od                   81
Name: count, dtype: int64

In [54]:
val_df['intent'].value_counts()

intent
support                2110
smokefree              1024
greetings               610
cravings                482
quitdate                426
noresponse              286
health                  282
fail                    258
nrt_itworks             205
weightgain              177
stress                  169
savingmoney             152
nrt_howtouse            118
cigsmell                109
nrt_don'twork           102
scared                   98
smokingless              95
nrt_dreams               86
ecigs                    68
nrt_stickissue           63
nrt_skinirritation       52
tiredness                51
nrt_mouthirritation      46
nrt_nauseous             41
nrt_od                   27
Name: count, dtype: int64

In [55]:
test['intent'].value_counts()

intent
support                1983
smokefree               818
greetings               490
quitdate                318
cravings                267
health                  265
noresponse              231
weightgain              167
fail                    152
stress                  148
nrt_itworks             135
nrt_howtouse            105
savingmoney              90
nrt_stickissue           83
scared                   83
cigsmell                 76
ecigs                    73
nrt_don'twork            72
nrt_dreams               54
tiredness                43
nrt_skinirritation       35
smokingless              33
nrt_nauseous             23
nrt_mouthirritation      21
nrt_od                   21
Name: count, dtype: int64